# AES - Step By Step

## Encryption

### Imports and Utility Functions

In [357]:
from tabulate import tabulate
import numpy as np

In [358]:
SBOX = {
    "00": "63", "01": "7c", "02": "77", "03": "7b", "04": "f2", "05": "6b", "06": "6f", "07": "c5",
    "08": "30", "09": "01", "0a": "67", "0b": "2b", "0c": "fe", "0d": "d7", "0e": "ab", "0f": "76",
    "10": "ca", "11": "82", "12": "c9", "13": "7d", "14": "fa", "15": "59", "16": "47", "17": "f0",
    "18": "ad", "19": "d4", "1a": "a2", "1b": "af", "1c": "9c", "1d": "a4", "1e": "72", "1f": "c0",
    "20": "b7", "21": "fd", "22": "93", "23": "26", "24": "36", "25": "3f", "26": "f7", "27": "cc",
    "28": "34", "29": "a5", "2a": "e5", "2b": "f1", "2c": "71", "2d": "d8", "2e": "31", "2f": "15",
    "30": "04", "31": "c7", "32": "23", "33": "c3", "34": "18", "35": "96", "36": "05", "37": "9a",
    "38": "07", "39": "12", "3a": "80", "3b": "e2", "3c": "eb", "3d": "27", "3e": "b2", "3f": "75",
    "40": "09", "41": "83", "42": "2c", "43": "1a", "44": "1b", "45": "6e", "46": "5a", "47": "a0",
    "48": "52", "49": "3b", "4a": "d6", "4b": "b3", "4c": "29", "4d": "e3", "4e": "2f", "4f": "84",
    "50": "53", "51": "d1", "52": "00", "53": "ed", "54": "20", "55": "fc", "56": "b1", "57": "5b",
    "58": "6a", "59": "cb", "5a": "be", "5b": "39", "5c": "4a", "5d": "4c", "5e": "58", "5f": "cf",
    "60": "d0", "61": "ef", "62": "aa", "63": "fb", "64": "43", "65": "4d", "66": "33", "67": "85",
    "68": "45", "69": "f9", "6a": "02", "6b": "7f", "6c": "50", "6d": "3c", "6e": "9f", "6f": "a8",
    "70": "51", "71": "a3", "72": "40", "73": "8f", "74": "92", "75": "9d", "76": "38", "77": "f5",
    "78": "bc", "79": "b6", "7a": "da", "7b": "21", "7c": "10", "7d": "ff", "7e": "f3", "7f": "d2",
    "80": "cd", "81": "0c", "82": "13", "83": "ec", "84": "5f", "85": "97", "86": "44", "87": "17",
    "88": "c4", "89": "a7", "8a": "7e", "8b": "3d", "8c": "64", "8d": "5d", "8e": "19", "8f": "73",
    "90": "60", "91": "81", "92": "4f", "93": "dc", "94": "22", "95": "2a", "96": "90", "97": "88",
    "98": "46", "99": "ee", "9a": "b8", "9b": "14", "9c": "de", "9d": "5e", "9e": "0b", "9f": "db",
    "a0": "e0", "a1": "32", "a2": "3a", "a3": "0a", "a4": "49", "a5": "06", "a6": "24", "a7": "5c",
    "a8": "c2", "a9": "d3", "aa": "ac", "ab": "62", "ac": "91", "ad": "95", "ae": "e4", "af": "79",
    "b0": "e7", "b1": "c8", "b2": "37", "b3": "6d", "b4": "8d", "b5": "d5", "b6": "4e", "b7": "a9",
    "b8": "6c", "b9": "56", "ba": "f4", "bb": "ea", "bc": "65", "bd": "7a", "be": "ae", "bf": "08",
    "c0": "ba", "c1": "78", "c2": "25", "c3": "2e", "c4": "1c", "c5": "a6", "c6": "b4", "c7": "c6",
    "c8": "e8", "c9": "dd", "ca": "74", "cb": "1f", "cc": "4b", "cd": "bd", "ce": "8b", "cf": "8a",
    "d0": "70", "d1": "3e", "d2": "b5", "d3": "66", "d4": "48", "d5": "03", "d6": "f6", "d7": "0e",
    "d8": "61", "d9": "35", "da": "57", "db": "b9", "dc": "86", "dd": "c1", "de": "1d", "df": "9e",
    "e0": "e1", "e1": "f8", "e2": "98", "e3": "11", "e4": "69", "e5": "d9", "e6": "8e", "e7": "94",
    "e8": "9b", "e9": "1e", "ea": "87", "eb": "e9", "ec": "ce", "ed": "55", "ee": "28", "ef": "df",
    "f0": "8c", "f1": "a1", "f2": "89", "f3": "0d", "f4": "bf", "f5": "e6", "f6": "42", "f7": "68",
    "f8": "41", "f9": "99", "fa": "2d", "fb": "0f", "fc": "b0", "fd": "54", "fe": "bb", "ff": "16"
}

mix_col_matrix = np.array([
    ['02','03','01','01'],
    ['01','02','03','01'],
    ['01','01','02','03'],
    ['03','01','01','02']
])

def string_to_hex(string):
    hex_string = ""
    for ch in string:
        hex_string += hex(ord(ch)).lstrip("0x")
    return hex_string

def hexstring_to_matrix(string):
    bytes = [string[i:i+2]for i in range (0, len(string), 2)]
    matrix = np.array(bytes).reshape(4, 4, order='F')
    return matrix

def rot_nib(nib):
    return nib[1:] + nib[:1]

def poly_to_binary(poly):
    poly = poly.polynomial()
    binary_rep = [poly.coefficient(i) for i in range(8)]
    return ''.join(str(b) for b in reversed(binary_rep))

def get_rcon(i):
    # Define the base field GF(2)
    F2 = GF(2)  
    
    # Define the polynomial ring over GF(2)
    R.<x> = PolynomialRing(F2)
    
    # Define the AES irreducible polynomial x^8 + x^3 + x + 1
    aes_poly = x^8 + x^4 + x^3 + x + 1  
    
    # Define the finite field GF(2^8) with AES irreducible polynomial
    F.<x> = GF(2**8, modulus=aes_poly)
    
    # Compute Rcon
    if i == 1:
        return poly_to_binary(x^0)  # 2^0 = 1
    else:
        return poly_to_binary(x^(i-1))  # 2^(i-1) in GF(2^8)

def hexlist_to_binarylist(hexlist):
    return [bin(int(h, 16))[2:].zfill(8) for h in hexlist]

def xor_binarylist(binary_list):
    result = []
    for i in range(8):
        bit_op = '0'
        for binary in binary_list:
            if binary[i] != bit_op:
                bit_op = '1'
            else:
                bit_op = '0'
        result.append(bit_op)

    return ''.join(result)

def xor_binarylists(list1, list2):
    xor_results = []

    for b1, b2 in zip(list1, list2):
        # Perform XOR operation bit by bit
        xor_result = ''.join('1' if b1[i] != b2[i] else '0' for i in range(len(b1)))
        xor_results.append(xor_result)
    
    return xor_results

def binarylist_to_hexlist(binary_values):
    return [hex(int(b, 2))[2:].zfill(2) for b in binary_values]

def gen_key_words(w):
    words = w
    for i in range(4, 44):
        if i%4 == 0:
            left_circular_shift_op = rot_nib(w[i-1])
            byte_substitution_op = [SBOX[byte] for byte in left_circular_shift_op]
            adding_round_const_op = xor_binarylists(hexlist_to_binarylist(byte_substitution_op), [get_rcon(i//4), '00000000', '00000000', '00000000'])
            g_op = adding_round_const_op
            word_op = binarylist_to_hexlist(xor_binarylists(hexlist_to_binarylist(w[i-4]), g_op))
            words.append(word_op)

        else:
            word_op = binarylist_to_hexlist(xor_binarylists(hexlist_to_binarylist(w[i-1]), hexlist_to_binarylist(w[i-4])))
            words.append(word_op)

    return words

def binary_to_poly(binary):
    R = PolynomialRing(GF(2), 'x')
    x = R.gen()
    poly = sum(int(bit) * x^i for i, bit in enumerate(reversed(binary)))
    return poly

#####################################
#           Round operations        #
#####################################
def add_roundkey(state_matrix_ip, round_no):
    state_matrix_ip_int = np.vectorize(lambda x: int(x, 16))(state_matrix_ip)
    round_key_int = np.vectorize(lambda x: int(x, 16))(round_keys[round_no])
    xor_result = np.bitwise_xor(state_matrix_ip_int, round_key_int)
    state_matrix_op = np.vectorize(lambda x: format(x, '02x'))(xor_result)
    print (f"\nAfter adding round key ({round_no}):")
    print(tabulate(state_matrix_op, tablefmt="grid"))
    return state_matrix_op

def byte_substitution(state_matrix_ip):
    state_matrix_op = np.vectorize(lambda x: SBOX.get(x, '00'))(state_matrix_ip)
    print("\nAfter byte substitution:")
    print(tabulate(state_matrix_op, tablefmt="grid"))
    return state_matrix_op

def shift_row(state_matrix_ip):
    state_matrix_op = np.vstack([state_matrix_ip[0], np.roll(state_matrix_ip[1], 3), np.roll(state_matrix_ip[2], 2), np.roll(state_matrix_ip[3], 1)])
    print("\nAfter Shift Row:")
    print(tabulate(state_matrix_op, tablefmt="grid"))
    return state_matrix_op

def mix_columns(state_matrix_ip):
    state_matrix_ip_bin = np.apply_along_axis(hexlist_to_binarylist, axis = 1, arr= state_matrix_ip)
    state_matrix_ip_poly = np.vectorize(binary_to_poly)(state_matrix_ip_bin)
    mix_col_matrix_bin = np.apply_along_axis(hexlist_to_binarylist, axis = 1, arr= mix_col_matrix)
    mix_col_matrix_poly = np.vectorize(binary_to_poly)(mix_col_matrix_bin)
    
    F2 = GF(2)  # Define GF(2)
    R.<x> = PolynomialRing(F2)
    aes_poly = x^8 + x^4 + x^3 + x + 1  # AES irreducible polynomial
    F.<a> = GF(2**8, modulus=aes_poly)  
    
    state_matrix_op_bin = []
    for i in range(4):
        rows = []
        for j in range(4):
            products = []
            for k in range(4):
                product = F(mix_col_matrix_poly[i, k]) * F(state_matrix_ip_poly[k, j])
                product_bin = poly_to_binary(product)
                products.append(product_bin)
            rows.append(xor_binarylist(products))
        state_matrix_op_bin.append(rows)
    
    state_matrix_op_bin = np.array(state_matrix_op_bin)
    
    # Convert binary values of state matrix to hex values
    state_matrix_op = np.apply_along_axis(binarylist_to_hexlist, axis = 1, arr = state_matrix_op_bin)
    print("\nAfter mix columns:")
    print(tabulate(state_matrix_op, tablefmt="grid"))
    return state_matrix_op

### Define the key (128 bits) and the plaintext (128 bits) to be encrypted

In [359]:
plaintext = "Two One Nine Two"
key = "Thats my Kung Fu"

### Get the hex values of the plaintext and the key

In [360]:
plaintext_hex = string_to_hex(plaintext)
key_hex = string_to_hex(key)
print("Plaintext (Hex): " + plaintext_hex)
print("Key (Hex): " + key_hex)

Plaintext (Hex): 54776f204f6e65204e696e652054776f
Key (Hex): 5468617473206d79204b756e67204675


### Arrange the key (hex) and plaintext (hex) into a matrix

In [366]:
plaintext_matrix = hexstring_to_matrix(plaintext_hex)
print("P =")
print(tabulate(plaintext_matrix, tablefmt="grid"))
print("\n")
key_m = hexstring_to_matrix(key_hex)
print("K 
=")
print(tabulate(key_m, tablefmt="grid"))

P = 

+----+----+----+----+
| 54 | 4f | 4e | 20 |
+----+----+----+----+
| 77 | 6e | 69 | 54 |
+----+----+----+----+
| 6f | 65 | 6e | 77 |
+----+----+----+----+
| 20 | 20 | 65 | 6f |
+----+----+----+----+


K = 

+----+----+----+----+
| 54 | 73 | 20 | 67 |
+----+----+----+----+
| 68 | 20 | 4b | 20 |
+----+----+----+----+
| 61 | 6d | 75 | 46 |
+----+----+----+----+
| 74 | 79 | 6e | 75 |
+----+----+----+----+


### Key Expansion: Generate all round keys

In [365]:
words = [key_m[:,0].tolist(), key_m[:,1].tolist(), key_m[:,2].tolist(), key_m[:,3].tolist()]

words = gen_key_words(words)

round_keys = []
for i in range(0, len(words), 4):
    print(f"\nKey_{i//4} = \n")
    k = np.array(w[i:i+4], dtype=str).T
    print(tabulate(k, tablefmt="grid"))
    
    round_keys.append(k)


Key_0 = 

+----+----+----+----+
| 54 | 73 | 20 | 67 |
+----+----+----+----+
| 68 | 20 | 4b | 20 |
+----+----+----+----+
| 61 | 6d | 75 | 46 |
+----+----+----+----+
| 74 | 79 | 6e | 75 |
+----+----+----+----+

Key_1 = 

+----+----+----+----+
| e2 | 91 | b1 | d6 |
+----+----+----+----+
| 32 | 12 | 59 | 79 |
+----+----+----+----+
| fc | 91 | e4 | a2 |
+----+----+----+----+
| f1 | 88 | e6 | 93 |
+----+----+----+----+

Key_2 = 

+----+----+----+----+
| 56 | c7 | 76 | a0 |
+----+----+----+----+
| 08 | 1a | 43 | 3a |
+----+----+----+----+
| 20 | b1 | 55 | f7 |
+----+----+----+----+
| 07 | 8f | 69 | fa |
+----+----+----+----+

Key_3 = 

+----+----+----+----+
| d2 | 15 | 63 | c3 |
+----+----+----+----+
| 60 | 7a | 39 | 03 |
+----+----+----+----+
| 0d | bc | e9 | 1e |
+----+----+----+----+
| e7 | 68 | 01 | fb |
+----+----+----+----+

Key_4 = 

+----+----+----+----+
| a1 | b4 | d7 | 14 |
+----+----+----+----+
| 12 | 68 | 51 | 52 |
+----+----+----+----+
| 02 | be | 57 | 49 |
+----+----+----+----+

### Add Round key (0)

In [363]:
state_matrix_0 = add_roundkey(plaintext_matrix, 0)


After adding round key (0):
+----+----+----+----+
| 00 | 3c | 6e | 47 |
+----+----+----+----+
| 1f | 4e | 22 | 74 |
+----+----+----+----+
| 0e | 08 | 1b | 31 |
+----+----+----+----+
| 54 | 59 | 0b | 1a |
+----+----+----+----+


### Perform Round Operations

In [364]:
for round_no in range(1, 11):
    print("\n" + '-'*20)
    print(f"Round {round_no}")
    print('-'*20)
    if round_no == 10:
        exec(f"state_matrix_{round_no} = add_roundkey(shift_row(byte_substitution(state_matrix_{round_no - 1})),{round_no})")
        print("\n" + '='*20)
        ciphertext = ''.join(state_matrix_10.flatten(order = 'F'))
        print(f"Ciphertext = {ciphertext}")
    else:
        exec(f"state_matrix_{round_no} = add_roundkey(mix_columns(shift_row(byte_substitution(state_matrix_{round_no - 1}))),{round_no})")


--------------------
Round 1
--------------------

After byte substitution:
+----+----+----+----+
| 63 | eb | 9f | a0 |
+----+----+----+----+
| c0 | 2f | 93 | 92 |
+----+----+----+----+
| ab | 30 | af | c7 |
+----+----+----+----+
| 20 | cb | 2b | a2 |
+----+----+----+----+

After Shift Row:
+----+----+----+----+
| 63 | eb | 9f | a0 |
+----+----+----+----+
| 2f | 93 | 92 | c0 |
+----+----+----+----+
| af | c7 | ab | 30 |
+----+----+----+----+
| a2 | 20 | cb | 2b |
+----+----+----+----+

After mix columns:
+----+----+----+----+
| ba | 84 | e8 | 1b |
+----+----+----+----+
| 75 | a4 | 8d | 40 |
+----+----+----+----+
| f4 | 8d | 06 | 7d |
+----+----+----+----+
| 7a | 32 | 0e | 5d |
+----+----+----+----+

After adding round key (1):
+----+----+----+----+
| 58 | 15 | 59 | cd |
+----+----+----+----+
| 47 | b6 | d4 | 39 |
+----+----+----+----+
| 08 | 1c | e2 | df |
+----+----+----+----+
| 8b | ba | e8 | ce |
+----+----+----+----+

--------------------
Round 2
--------------------

After byte s